# SIADS 516 Homework 4: Spark SQL
Version 1.0.20200221.1
### Dr. Chris Teplovs, School of Information, University of Michigan
<small><a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" /></a>This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>.

In [1]:
"""
NAME: Paul Natland
"""

'\nNAME: Paul Natland\n'

------------------------------------------------------------------------------------------------------------------------------
#### <font color="blue">__ACKNOWLEDGEMENTS:__ I benefitted from class notebooks and lectures, searches on Stack Overflow, and conversations in SLACK</font>
------------------------------------------------------------------------------------------------------------------------------

This homework assignment uses the Yelp Academic dataset, with which you should now be familiar.
We have created a few cells to get you started, but you're largely on your own to devise solutions to the
"real-world" questions below.

The best solutions will use spark.sql() calls as a preferred way to query the dataset and also use the fewest number of steps.  For example, to find the answer to "How many users have more than 100 "cool" votes?", this:
```
query = """
SELECT count(*) FROM user WHERE cool > 100
"""
spark.sql(query).show()
```
is preferable to:
```
user.filter('cool > 100').show()
```
or 
```
query = """
SELECT * FROM user
"""
df = spark.sql(query)
df.filter('cool > 100').show()
```
(Note that the last number is somewhat ridiculous.)

## Imports

In [2]:
import re
    #to use regex for word extraction
    
from pyspark.sql import SparkSession
    #to establish the sparksession
    
from pyspark.sql.functions import udf
    #to call fonctions on columns

from pyspark.sql.types import IntegerType, StringType, ArrayType
    #to explicitly state output value types in udf's

Our usual Spark mantra:

In [3]:
spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName('SIADS 516 -- HW4') \
    .getOrCreate() 

sc = spark.sparkContext

Load the JSON files:

In [4]:
business = spark.read.json('data/yelp_academic/yelp_academic_dataset_business.json.gz')
checkin = spark.read.json('data/yelp_academic/yelp_academic_dataset_checkin.json.gz')
review = spark.read.json('data/yelp_academic/yelp_academic_dataset_review.json.gz')
tip = spark.read.json('data/yelp_academic/yelp_academic_dataset_tip.json.gz')
user = spark.read.json('data/yelp_academic/yelp_academic_dataset_user.json.gz')

Create temp views for the DataFrames:

In [5]:
business.createOrReplaceTempView("business")
checkin.createOrReplaceTempView("checkin")
tip.createOrReplaceTempView("tip")
review.createOrReplaceTempView("review")
user.createOrReplaceTempView("user")

## <font color="magenta">Q1. How many users have more than 500 fans?</font>

In [6]:
query_1 = \
"""SELECT name,fans FROM user WHERE fans > 500"""
    
spark.sql(query_1).count()

185

In [7]:
#sampling the query to check it yields the correct results
spark.sql(query_1).show(10)

+-------+----+
|   name|fans|
+-------+----+
|  Keane| 696|
|Michael| 912|
|    Ann|1055|
|  Scott| 587|
|   Elsa| 649|
| Denise| 545|
|   Jack|1500|
|   Dave|1010|
|Jessica| 544|
|   Cara|1150|
+-------+----+
only showing top 10 rows



## <font color="magenta">Q2. How many businesses from Madison, Wisconsin are represented in the dataset?</font>

In [8]:
query_2 = \
""" 
    SELECT name,city,state 
    FROM business 
    WHERE city = 'Madison' and state = 'WI'
"""
    
spark.sql(query_2).count()

3493

In [9]:
#sampling the query to check it yields the correct results
spark.sql(query_2).show(5)

+--------------------+-------+-----+
|                name|   city|state|
+--------------------+-------+-----+
|     Kirsty Blattner|Madison|   WI|
|       QBE Insurance|Madison|   WI|
|Habitat For Human...|Madison|   WI|
|             Wendy's|Madison|   WI|
| Susan Hughes Design|Madison|   WI|
+--------------------+-------+-----+
only showing top 5 rows



## <font color="magenta">Q3: How many users have more than 500 fans?</font>

> __SEE Q1 :)__

## <font color="magenta">Q4: Which US states are represented in the data set?  Use full names of states.</font>

In [10]:
states = {"AL":"Alabama","AK":"Alaska","AZ":"Arizona","AR":"Arkansas","CA":"California","CO":"Colorado",
          "CT":"Connecticut","DE":"Delaware","FL":"Florida","GA":"Georgia","HI":"Hawaii","ID":"Idaho",
          "IL":"Illinois","IN":"Indiana","IA":"Iowa","KS":"Kansas","KY":"Kentucky","LA":"Louisiana",
          "ME":"Maine","MD":"Maryland","MA":"Massachusetts","MI":"Michigan","MN":"Minnesota","MS":"Mississippi",
          "MO":"Missouri","MT":"Montana","NE":"Nebraska","NV":"Nevada","NH":"New Hampshire","NJ":"New Jersey",
          "NM":"New Mexico","NY":"New York","NC":"North Carolina","ND":"North Dakota","OH":"Ohio","OK":"Oklahoma",
          "OR":"Oregon","PA":"Pennsylvania","RI":"Rhode Island","SC":"South Carolina","SD":"South Dakota",
          "TN":"Tennessee","TX":"Texas","UT":"Utah","VT":"Vermont","VA":"Virginia","WA":"Washington",
          "WV":"West Virginia","WI":"Wisconsin","WY":"Wyoming"}

In [11]:
query_4= \
    """ 
        SELECT DISTINCT state FROM business
    """
    
states_included = spark.sql(query_4).toPandas()

In [12]:
(states_included \
     #excludes abbreviations that don't correspond to US states
     [states_included.state.isin(states.keys()) == True]
 
     #uses the dictionary 'states' to replace the abbreviations with full state names
     .replace({"state": states})
 
     #an alphabetical sort for fun =)
     .sort_values('state')
)

,state
17,Alabama
33,Alaska
0,Arizona
34,Arkansas
9,California
11,Connecticut
32,Florida
31,Georgia
14,Illinois
10,Nebraska


## <font color="magenta">Q5: What is the text of the funniest review?</font>

In [13]:
query_5 = \
""" 
    SELECT text
    FROM review
    ORDER BY funny DESC
"""
    
spark.sql(query_5).first()

Row(text="Flew to Arizona a few months ago to try this. Disappointed that this place closed after Gordon Ramsay's return. Their food was one of the best food I had in my whole entire life!! Their caesar salad was great, sauce is very good. Their pizza was THE BEST I HAD IN MY WHOLE ENTIRE LIFE. From the garlic crust to the extremely cheesy topping, the pizza was absolutely outstanding. Wish I could come back but its now closed.")

## <font color="magenta">Q6: Which review(s) has/have the most words?  What do you notice about the maximum word count?</font>

In [14]:
#function created to count words (I chose to leave out numeric characters)
def word_count(text):
    """return the count of the number of words in the input text"""
    return len(re.findall(r'[A-Za-z\']+', text.lower()))

#creation of user defined function from function above
word_count_udf = udf(lambda line: word_count(line), IntegerType())

#register use of udf so that it can be used in the sql query
spark.udf.register('word_count_udf', word_count, IntegerType())

<function __main__.word_count(text)>

In [15]:
query_6 = \
""" 
    SELECT 
        text,
        word_count_udf(text) AS review_word_count,
        LENGTH(text) AS review_character_count
    FROM review
    ORDER BY review_word_count DESC
"""
    
spark.sql(query_6).show(10)

+--------------------+-----------------+----------------------+
|                text|review_word_count|review_character_count|
+--------------------+-----------------+----------------------+
|Went in Sunday 12...|             1043|                  5000|
|This place is HOR...|             1041|                  4990|
|Where do I even s...|             1037|                  5000|
|If I could negati...|             1036|                  4982|
|I made my appoint...|             1033|                  4958|
|PART ONE: As of n...|             1033|                  4963|
|They asked me to ...|             1032|                  4997|
|We stayed here th...|             1028|                  5000|
|NOW THAT'S A KNIF...|             1026|                  4987|
|If i could i woul...|             1021|                  4999|
+--------------------+-----------------+----------------------+
only showing top 10 rows



 > There are a few things I notice: first, the reviews that have the highest word count (and I chose to exclude items in the review that contained numbers) all have similar word counts (just over 1000).  Finding the character count for those reviews (shown in the aliased column <code>review_character_count</code>) you find the character count hovers around 5000 for each (which, according to [links on yelp.com like this one](https://www.yelp.com/topic/nampa-whats-the-character-word-limit-allowed-for-reviews), is the maximum number of characters allowed).

##  <font color="magenta">Q7: What are the names of the top 10 users who provided the most tips?</font>

In [16]:
query_7 = \
""" 
    SELECT 
        user.name, 
        count(tip.user_id) AS tip_count
    FROM tip
    INNER JOIN user
        ON tip.user_id = user.user_id
    GROUP BY 
        tip.user_id, 
        user.name
    ORDER BY tip_count DESC
"""
    
spark.sql(query_7).show(10)

+--------+---------+
|    name|tip_count|
+--------+---------+
|    Momo|     2439|
|Jennifer|     1598|
|Samantha|     1509|
|  Daniel|     1376|
|Christie|     1352|
|     May|     1255|
|   Kurdy|     1178|
| Anthony|     1161|
| Shirley|     1154|
| Cherrie|     1017|
+--------+---------+
only showing top 10 rows



##  <font color="magenta">Q8: List the names, number of reviews of businesses in Arizona and total number of reviews of the top 5 users (as determined by who has created the most number of reviews of businesses in Arizona).  Include a column that shows the percentage of reviews that are of businesses from Arizona.  The first row of the results should be:</font>
```
+--------+--------+-----------+---------+
|    name|az_count|total_count|  percent|
+--------+--------+-----------+---------+
|    Brad|    1637|       1642|99.695496|
+--------+--------+-----------+---------+
```

In [17]:
query_8 = \
""" 
    SELECT
       name,
       az_count,
       total_count,
       ROUND(az_count*100/total_count,6) 
           AS percent
    FROM
        (
         SELECT 
            user.name,
            user.user_id,
            sum(cast(business.state = 'AZ' as INT)) 
                AS az_count,
            user.review_count 
                AS total_count
        FROM user
        INNER JOIN review
            ON review.user_id = user.user_id
        INNER JOIN business
            ON business.business_id = review.business_id
        GROUP BY 
            user.name,
            user.user_id,
            total_count
        ORDER BY az_count DESC
        
        ) resultset
"""
    
spark.sql(query_8).show(5)

+--------+--------+-----------+---------+
|    name|az_count|total_count|  percent|
+--------+--------+-----------+---------+
|    Brad|    1637|       1642|99.695493|
|   Karen|    1559|       2340|66.623932|
|Jennifer|    1250|       1929|64.800415|
|    Gabi|    1151|       1932|59.575569|
|Jennifer|    1059|       4190|25.274463|
+--------+--------+-----------+---------+
only showing top 5 rows

